## CV Parking Lot Analysis

#### Date: 9/30/2025
#### Author: Nineveh O'Connell

Goal: This notebook ingests csv output of computer vision model of the willow creek trailhead and output length of stay/turnover metrics.

In [29]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta



Define parking spot outlines (spot_id, minx, maxy, miny) and sort by minx

In [30]:

spot_boundaries = pd.DataFrame({
    "spot_id": np.arange(1, 17),
    "minx": [640, 549, 490, 430, 380, 330, 280, 240, 195, 165, 140, 112, 102, 75, 40, 5],
    "maxy": [340, 340, 340, 340, 335, 325, 320, 320, 330, 320, 315, 310, 305, 300, 295, 295],
    "miny": [265] * 16
}).sort_values("minx").reset_index(drop=True)


Read in data

In [31]:
file_ts = "2025-10-01 20-39-35"

# 2) read in data (adjust path as needed)
csv_path = "/Users/Nineveh.OConnell/OneDrive - DOT OST/volpe-portfolio-PublicLands - AI Real-Time Parking Project/Data/data_download_export_" + file_ts + ".csv"
in_willow_creek = pd.read_csv(csv_path)


Assign to parking spots, calling the parking spot -1 in the case of being in the roadway

In [32]:
in_willow_creek['parking_spot_id'] = pd.cut(in_willow_creek['cx'], bins = spot_boundaries['minx'], labels = np.arange(1,16)[::-1], right = True)

# if midpoint has y higher (below) than 335, mark it as being in the roadway
in_willow_creek['parking_spot_id'] = in_willow_creek['parking_spot_id'].cat.add_categories('-1')
in_willow_creek.loc[in_willow_creek['cy'] > 335, 'parking_spot_id'] = '-1'

Make confidence into a numeric variable and only keep instances with confidence over 0.35. From spot checking, instances with lower confidence are not really vehicles.

In [33]:
# 6) extract numeric confidence from a string like "label (0.82)" into confidence_numeric
#    regex captures the number inside parentheses (first occurrence)
def extract_confidence(s):
    if pd.isna(s):
        return np.nan
    m = re.search(r"\(([^)]+)\)", str(s))
    if m:
        try:
            return float(m.group(1))
        except ValueError:
            return np.nan
    return np.nan

in_willow_creek["confidence_numeric"] = in_willow_creek["confidence"].apply(extract_confidence)

# keep only rows where confidence is 0.35 or greater
willow_creek_vehicles = in_willow_creek[in_willow_creek["confidence_numeric"] > 0.35]
# Clean up column
willow_creek_vehicles = willow_creek_vehicles.drop(columns=["confidence"])
# make timestamp actual date time object, and turn id into a string
willow_creek_vehicles['timestamp_dt'] = pd.to_datetime(willow_creek_vehicles['timestamp'], format='%Y-%m-%d %H-%M-%S')



In cleaning up data resulting from computer vision of vehicles in the parking lot, I have noticed that sometimes the computer vision model re-recognized a vehicle after too many frames of the id missing, it wasn’t feasibly the same vehicle. In the future we can toggle this in the tracker itself by setting a track_buffer threshold. For now, the code below pulls out the unique and comprehensive list of timestamps within, and adds a unique suffix to future records of an id if the same id appears again in the dataset after missing from the data for 5 or more frames.

In [34]:
def split_long_gaps_and_suffix_ids(df,
                                   id_col="id",
                                   ts_col="timestamp",
                                   gap_threshold=25,
                                   inplace=False):
    if not inplace:
        df = df.copy()

    # *** NEW: Drop IDs which appear only once in the unique timestamps ***
    id_counts = df.groupby(id_col)[ts_col].nunique()
    valid_ids = id_counts[id_counts > 1].index
    df = df[df[id_col].isin(valid_ids)].copy()

    # Build sorted unique timestamps and mapping timestamp -> frame index
    unique_ts = pd.Index(sorted(df[ts_col].unique()))
    ts_to_index = {ts: idx for idx, ts in enumerate(unique_ts)}

    df["_frame_index"] = df[ts_col].map(ts_to_index)
    df = df.sort_values([id_col, "_frame_index"]).reset_index(drop=True)

    new_ids = []
    state = {}

    for _, row in df.iterrows():
        orig_id = row[id_col]
        fi = int(row["_frame_index"])

        if orig_id not in state:
            state[orig_id] = {"last_idx": fi, "counter": 0}
            new_ids.append(orig_id)
            continue

        last_idx = state[orig_id]["last_idx"]
        gap = fi - last_idx

        if gap >= gap_threshold:
            state[orig_id]["counter"] += 1
            suffix = state[orig_id]["counter"]
            new_id = f"{orig_id}_{suffix}"
            state[orig_id]["last_idx"] = fi
            new_ids.append(new_id)
        else:
            cnt = state[orig_id]["counter"]
            if cnt == 0:
                new_ids.append(orig_id)
            else:
                new_ids.append(f"{orig_id}_{cnt}")
            state[orig_id]["last_idx"] = fi

    df["id_suffixed"] = new_ids
    df = df.sort_index()
    df = df.drop(columns=["_frame_index"])

    return df, unique_ts

# use above function
if __name__ == "__main__":
    willow_creek_vehicles, unique_ts = split_long_gaps_and_suffix_ids(willow_creek_vehicles)

In [35]:
# parameters
tolerance = 2                # pixels; consider duplicates when |dx| <= tolerance and |dy| <= tolerance

# will collect kept rows
kept_indices = []

# group by timestamp and parking_spot_id
group_cols = ["timestamp", "parking_spot_id"]

# iterate groups — this is efficient unless you have extremely many tiny groups
for (ts_val, spot_val), group in willow_creek_vehicles.groupby(group_cols, sort=False):

    # convert group to numpy arrays for speed
    # sort by confidence desc so first kept are the best
    order = np.argsort(-group["confidence_numeric"])
    group_idx = group.index.to_numpy()[order]
    cx_arr = group.loc[group_idx, "cx"].to_numpy()
    cy_arr = group.loc[group_idx, "cy"].to_numpy()

    # keep list of indices for this group
    kept_for_group = []

    # iterate detections in descending confidence order
    for i, idx in enumerate(group_idx):
        cx_i = cx_arr[i]
        cy_i = cy_arr[i]

        # if cx or cy is NaN, treat as not matching any kept point (so it can be kept only if best)
        if np.isnan(cx_i) or np.isnan(cy_i):
            # if the best entry is NaN and there are others non-NaN, this NaN will still be kept only
            # if it has the highest confidence; follow the same logic as R code where exact matches matter
            # Here, proceed to keep if no kept point exists (or if it's highest confidence)
            if len(kept_for_group) == 0:
                kept_for_group.append(idx)
            continue

        # check against already-kept points: if any kept point is within tolerance in both x and y, skip
        if kept_for_group:
            kept_cx = willow_creek_vehicles.loc[kept_for_group, "cx"].to_numpy()
            kept_cy = willow_creek_vehicles.loc[kept_for_group, "cy"].to_numpy()

            # compute boolean mask of kept points within tolerance (|dx| <= tol and |dy| <= tol)
            # using broadcasting
            dx = np.abs(kept_cx - cx_i)
            dy = np.abs(kept_cy - cy_i)
            within_tol = (dx <= tolerance) & (dy <= tolerance)

            if np.any(within_tol):
                # a kept point already within tolerance — consider current row a duplicate -> skip
                continue

        # otherwise, keep this detection
        kept_for_group.append(idx)

    # extend global kept list
    kept_indices.extend(kept_for_group)

# create deduplicated DataFrame preserving original relative order of kept rows
kept_mask = willow_creek_vehicles.index.isin(kept_indices)
willow_creek_vehicles = willow_creek_vehicles.loc[kept_mask].copy()


C:\Users\Nineveh.OConnell\AppData\Local\Temp\ipykernel_18324\2009858674.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (ts_val, spot_val), group in willow_creek_vehicles.groupby(group_cols, sort=False):


Ok, now I feel good about the general confidence level and deduplicating we've applied. There may be more errors to catch, but this way hopefully a single vehicle should be recognized for longer as being in the same place. By vehicle id and class and parking spot, let's capture the earliest time it was seen and last time it was seen. Perhaps let's also capture the lowest confidence and highest confidence

In [36]:

# create concatenated string uniquely identifying vehicles to make my life easier
willow_creek_vehicles['id_string'] = willow_creek_vehicles['id_suffixed'].astype(str)
willow_creek_vehicles['parking_spot_id_str'] = willow_creek_vehicles['parking_spot_id'].astype(str)

# output min and max time stamps for each vehicle
vehicle_time_boundaries = willow_creek_vehicles.groupby('id_string')['timestamp_dt'].agg(['min', 'max', 'count'])
vehicle_confidence_boundaries = willow_creek_vehicles.groupby('id_string')['confidence_numeric'].agg(['min', 'max', 'median'])
vehicle_parking_spot_list = willow_creek_vehicles.groupby('id_string')['parking_spot_id_str'].agg(lambda x: ','.join(x.unique().astype(str)))

# merge together
unique_vehicles_summary = pd.merge(vehicle_time_boundaries, vehicle_confidence_boundaries, on = 'id_string', suffixes= ("_time", "_conf"))
unique_vehicles_summary = pd.merge(willow_creek_vehicles[['id_string']].drop_duplicates(), unique_vehicles_summary, on = 'id_string')
unique_vehicles_summary = pd.merge(unique_vehicles_summary, vehicle_parking_spot_list, on = 'id_string')

In [37]:
# calculate length of stay
unique_vehicles_summary['length_of_stay_minutes'] = (unique_vehicles_summary['max_time'] - unique_vehicles_summary['min_time']).dt.total_seconds()/60

# remove additional entries with overall low confidence
unique_vehicles_summary = unique_vehicles_summary[(unique_vehicles_summary['max_conf'] > 0.6) & (unique_vehicles_summary['median'] > 0.5)]

# and remove entries with less than two minutes of recognition -- I don't trust the cv like that, this only happened when I was pulling the same frame multiple times before
unique_vehicles_summary = unique_vehicles_summary[(unique_vehicles_summary['length_of_stay_minutes'] > 1.5)]


Additional error resolution is needed, as vehicles may be identified incorrectly multiple times. If one parking spot has multiple vehicles recognized as being present with overlapping time bands, keep only the record with the wider timeband. This should also work by identifying the record with the most counts, but you'd have to identify the time band anyway to know which to remove -- so let's apply this more rigorous approach.

In [38]:
def keep_widest_nonoverlapping(df, spot_col='parking_spot_id_str', t0_col='min_time', t1_col='max_time'):

    kept_indices = []

    # process per spot
    for spot, group in df.groupby(spot_col, sort=False):
        # sort by duration desc, stable so ties preserve original order
        group_sorted = group.sort_values('length_of_stay_minutes', ascending=False, kind='stable')

        kept_for_spot = []
        kept_intervals = []  # list of (min_time, max_time) for kept intervals

        for idx, row in group_sorted.iterrows():
            t0 = row[t0_col]
            t1 = row[t1_col]
            # check overlap with any already-kept interval
            # overlap condition: not (t1 <= kept_t0 or t0 >= kept_t1)
            overlaps = any((t1 > kt0) and (t0 < kt1) for (kt0, kt1) in kept_intervals)
            if not overlaps:
                kept_for_spot.append(idx)
                kept_intervals.append((t0, t1))

        kept_indices.extend(kept_for_spot)

    # return filtered dataframe
    result = df.loc[kept_indices]
    return result

# deduplicate nested time spans
unique_vehicles_summary_dedup = keep_widest_nonoverlapping(unique_vehicles_summary,
                                                         spot_col='parking_spot_id_str',
                                                         t0_col='min_time',
                                                         t1_col='max_time')


Ok, great. We end up with a similar number of vehicles to my manual count -- this indicates 14 vehicles (seeing as one is just the truck that was passing by) and I manually counted 15 vehicles over this time period. Vehicles should be unique by id -- the class can vary -- and the parking spot identification is tricky and not requisite to deem a vehicle as uniquely identified, though it was useful for deduplication because I specifically looked at nesting time frames (overlapping, non-nested time frames should be permitted). Length of stay should ideally exclude vehicles that were there at the beginning and those that were still there at the end.

In [39]:
cv_start_time = unique_vehicles_summary_dedup['min_time'].agg('min')
cv_end_time = unique_vehicles_summary_dedup['max_time'].agg('max')

# flag as present start of period if so, end of period do the same. allow two minute buffer because sometimes a vehilce is mixed in a given frame
unique_vehicles_summary_dedup['flag_present_start'] = (unique_vehicles_summary_dedup['min_time'] - timedelta(minutes=2)) < cv_start_time
unique_vehicles_summary_dedup['flag_present_end']   = (unique_vehicles_summary_dedup['max_time'] + timedelta(minutes=2)) > cv_end_time



In [40]:
# for this operation, drop vehicles that were not in a parking spot
parked_cars_only = unique_vehicles_summary_dedup[unique_vehicles_summary_dedup['parking_spot_id_str'] != '-1']

# summarize length of stay by unique car, ignoring spot now
parked_cars_only = parked_cars_only.groupby('id_string')[['length_of_stay_minutes', 'flag_present_start', 'flag_present_end']].agg('sum')



In [41]:
# count overall unique vehicles in the dataset
n_parked_in_period = len(parked_cars_only)
# count unique vehicles present at the start
n_parked_at_start = sum(parked_cars_only['flag_present_start'] == 1)
# count unique vehicles present at the end
n_parked_at_end = sum(parked_cars_only['flag_present_end'] == 1)
# count unique vehicles for which we witnessed the beginning and end
n_whole_stay_witnessed = len(parked_cars_only[(parked_cars_only['flag_present_end'] == 0) & (parked_cars_only['flag_present_start'] == 0)])
# average length of stay
avg_length_of_stay = parked_cars_only[(parked_cars_only['flag_present_end'] == 0) & (parked_cars_only['flag_present_start'] == 0)]['length_of_stay_minutes'].agg('mean')
# print all those things
print("Between ", cv_start_time, " and ", cv_end_time, ", ", n_parked_in_period, " total vehicles were captured at the Willow Creek Trailhead.\n", 
      n_parked_at_start, " were already parked when this time period began and ", n_parked_at_end, " were present when this time period ended. \n " \
      "We witnessed the beginning and end of the visit to the trailhead for ",
      n_whole_stay_witnessed, " vehicles. The average stay for these vehicles was ", avg_length_of_stay, " minutes.")

Between  2025-10-01 17:49:46  and  2025-10-01 20:39:35 ,  17  total vehicles were captured at the Willow Creek Trailhead.
 7  were already parked when this time period began and  9  were present when this time period ended. 
 We witnessed the beginning and end of the visit to the trailhead for  3  vehicles. The average stay for these vehicles was  48.36111111111111  minutes.


In [42]:
# save unique vehicle info in post processing folder
unique_vehicles_summary_dedup.to_csv('/Users/Nineveh.OConnell/OneDrive - DOT OST/volpe-portfolio-PublicLands - AI Real-Time Parking Project/Postprocessing/willow_creek_parking_summary_' + file_ts + '.csv', index=False)